In [1]:
TEST = False
DROPNA = True
NSUBS = 10000
TESTSUBS = 10000
MINSTUDENTS = 30
MINITEMS = 30
ADD_ABILITIES = False

In [2]:
import pyreadr
import os
import pandas as pd
from datetime import datetime
import numpy as np
from utils import mymode
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

Definitions

COLS = [
    'studentId',
    'assessmentId',
    'class',
    'code',
    'grade',
    'Gender',
    'candidateResponses',
    'useCase',
    'motherTongue',
    'age',
    'scale',
    'timestamp',
    'timestamp_2',
    'timeOpened',
    'timeClosed',
    'viewingTime',
    'matrix',
    'IRT_difficulty',
    'score'
  ]


In [3]:
STUDENT_COLS = ['studentId', 'motherTongue', 'Gender']
ITEM_COLS = ['code', 'scale', 'matrix', 'IRT_difficulty', 'topic', 'responseformat', 'textlength']
EDGE_COLS = ['score', 'age', 'grade', 'viewingTime', 'timestamp', 'useCase', 'assessmentId', 
            'school_location', 'school_postcode', 'canton', 'school_code'] #, 'timestamp'

MAINCOLS = ['studentId', 'motherTongue', 'Gender', 'code', 'scale', 'score', 'age', 'grade'] 

DATA_PATH = './data/dd_2024_Jul_valid_parameters_schools.rds'
TEST_PATH = './data/dd_2024_Jul_valid_test_%d.csv'%TESTSUBS
OUT_PATH = './data/mindsteps_set_2024'
#TEST_PATH = './data/dd_2023_Jan_valid_test_5000.csv'

Load and prepare data

In [4]:
if not TEST:
    rds = pyreadr.read_r(DATA_PATH)
    df = rds[None] 
    print(df.shape)
    if False:
        studentIds = np.random.choice(df['studentId'].unique(), TESTSUBS, replace=False)
        df_sel = df.loc[df.studentId.isin(studentIds)]
        df_sel.to_csv(TEST_PATH)
else:
    df = pd.read_csv(TEST_PATH, low_memory=False)

(46032740, 45)


In [6]:
print(df.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46032740 entries, 0 to 46032739
Data columns (total 45 columns):
 #   Column              Dtype         
---  ------              -----         
 0   school_code         object        
 1   code                object        
 2   grade               int32         
 3   studentId           int32         
 4   timestamp           float64       
 5   age                 float64       
 6   assessmentId        object        
 7   class               object        
 8   Gender              object        
 9   candidateResponses  object        
 10  useCase             object        
 11  motherTongue        int32         
 12  scale               object        
 13  timeOpened          object        
 14  timeClosed          object        
 15  viewingTime         float64       
 16  itemOrder           object        
 17  score               float64       
 18  ILZ_D               int32         
 19  ILZ_E               int32         
 20  

/home/garben/anaconda3/GNN/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:403: RuntimeWarning: invalid value encountered in cast
  new_result = result.astype(dtype)
/home/garben/anaconda3/GNN/lib/python3.8/site-packages/numpy/core/numeric.py:2388: RuntimeWarning: invalid value encountered in multiply
  y = y * ones_like(cond)
/home/garben/anaconda3/GNN/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:403: RuntimeWarning: invalid value encountered in cast
  new_result = result.astype(dtype)
/home/garben/anaconda3/GNN/lib/python3.8/site-packages/numpy/core/numeric.py:2388: RuntimeWarning: invalid value encountered in multiply
  y = y * ones_like(cond)
/home/garben/anaconda3/GNN/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:403: RuntimeWarning: invalid value encountered in cast
  new_result = result.astype(dtype)
/home/garben/anaconda3/GNN/lib/python3.8/site-packages/numpy/core/numeric.py:2388: RuntimeWarning: invalid value encountered in multiply
  y = y * ones_lik

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46032740 entries, 0 to 46032739
Data columns (total 45 columns):
 #   Column              Dtype         
---  ------              -----         
 0   school_code         object        
 1   code                object        
 2   grade               int8          
 3   studentId           int32         
 4   timestamp           float32       
 5   age                 float32       
 6   assessmentId        object        
 7   class               object        
 8   Gender              object        
 9   candidateResponses  object        
 10  useCase             object        
 11  motherTongue        int8          
 12  scale               object        
 13  timeOpened          object        
 14  timeClosed          object        
 15  viewingTime         float64       
 16  itemOrder           object        
 17  score               float32       
 18  ILZ_D               int8          
 19  ILZ_E               int8          
 20  

In [7]:
try:
    df['day'] = df['date'].dt.strftime('%Y-%m-%d')        
except:
    df['day'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d'))
            
print(df.columns)
#df['day'] = df['date'].dt.strftime('%Y-%m-%d')

#df['timestamp'] = df['timestamp_2'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S'))
#df = df.sort_values(by=['studentId', 'assessmentId', 'timestamp'])
#df_grouped = df.groupby(by=['studentId', 'assessmentId'])['timestamp'].nunique()
#print((df_grouped > 1).mean())
#df['code_previous'] = df.groupby('studentId')['code'].transform(lambda x: x.shift(1))
#print(df.head(20))
print(df.columns)
print(df.shape)
print(df.isnull().sum())

Index(['school_code', 'code', 'grade', 'studentId', 'timestamp', 'age',
       'assessmentId', 'class', 'Gender', 'candidateResponses', 'useCase',
       'motherTongue', 'scale', 'timeOpened', 'timeClosed', 'viewingTime',
       'itemOrder', 'score', 'ILZ_D', 'ILZ_E', 'ILZ_F', 'ILZ_MA', 'ILZ_N',
       'date', 'year', 'previous_sessions', 'years_from_start', 'pace',
       'frequency', 'IRT_difficulty', 'discrimination', 'matrix', 'topic',
       'task_type', 'item_type', 'difficulty', 'responseformat',
       'numberofchoices', 'audiolength', 'textlength', 'nchar', 'nwords',
       'canton', 'school_postcode', 'school_location', 'day'],
      dtype='object')
Index(['school_code', 'code', 'grade', 'studentId', 'timestamp', 'age',
       'assessmentId', 'class', 'Gender', 'candidateResponses', 'useCase',
       'motherTongue', 'scale', 'timeOpened', 'timeClosed', 'viewingTime',
       'itemOrder', 'score', 'ILZ_D', 'ILZ_E', 'ILZ_F', 'ILZ_MA', 'ILZ_N',
       'date', 'year', 'previous_se

In [8]:
DATA_PATH

'./data/dd_2024_Jul_valid_parameters_schools.rds'

In [9]:
df.columns

Index(['school_code', 'code', 'grade', 'studentId', 'timestamp', 'age',
       'assessmentId', 'class', 'Gender', 'candidateResponses', 'useCase',
       'motherTongue', 'scale', 'timeOpened', 'timeClosed', 'viewingTime',
       'itemOrder', 'score', 'ILZ_D', 'ILZ_E', 'ILZ_F', 'ILZ_MA', 'ILZ_N',
       'date', 'year', 'previous_sessions', 'years_from_start', 'pace',
       'frequency', 'IRT_difficulty', 'discrimination', 'matrix', 'topic',
       'task_type', 'item_type', 'difficulty', 'responseformat',
       'numberofchoices', 'audiolength', 'textlength', 'nchar', 'nwords',
       'canton', 'school_postcode', 'school_location', 'day'],
      dtype='object')

In [10]:
%matplotlib inline
#df.grade.plot.hist()
#plt.show()

In [11]:
# Add abilities
if ADD_ABILITIES:
    MS_PATH = './data/abilities_2pl.rda'
    rds = pyreadr.read_r(MS_PATH)
    df_abilities = rds['abilities']
    df_abilities['studentId'] = df_abilities['studentId'].astype(int)
    df_abilities = df_abilities.rename(columns={'Ability': 'ability', 'Scale':'scale'})
    df_abilities = df_abilities.loc[df_abilities.model == 'all_2_studentId_personpreds_itempreds_excludeBadSessions_onlyClosed']
    df_abilities['day'] = df_abilities['timestamp'].dt.strftime('%Y-%m-%d')
    df_abilities = df_abilities[['studentId', 'day', 'scale', 'ability']]

    df.set_index(['studentId', 'scale', 'day'], inplace=True)
    df_abilities.set_index(['studentId', 'scale', 'day'], inplace=True)
    df = df.join(df_abilities, how='left').reset_index()
    #df = df.merge(df_abilities, on=['studentId', 'scale', 'day'], how='left').reset_index()
    EDGE_COLS = EDGE_COLS + ['ability']


COLS = STUDENT_COLS + ITEM_COLS + EDGE_COLS
print(COLS)

['studentId', 'motherTongue', 'Gender', 'code', 'scale', 'matrix', 'IRT_difficulty', 'topic', 'responseformat', 'textlength', 'score', 'age', 'grade', 'viewingTime', 'timestamp', 'useCase', 'assessmentId', 'school_location', 'school_postcode', 'canton', 'school_code']


In [12]:
# student attributes    
df_student = df[STUDENT_COLS].groupby(by=['studentId']).agg(mymode).reset_index() 

# item attributes
df_item = df[ITEM_COLS].groupby(by=['code']).agg(mymode).reset_index()

#df = df.merge(df_student, on='studentId', suffixes = ('_x', ''))

df.set_index('studentId', inplace=True)
df_student.set_index('studentId', inplace=True)
df = df.join(df_student, how='inner', lsuffix ='_x').reset_index()

In [13]:
df.columns

Index(['studentId', 'school_code', 'code', 'grade', 'timestamp', 'age',
       'assessmentId', 'class', 'Gender_x', 'candidateResponses', 'useCase',
       'motherTongue_x', 'scale', 'timeOpened', 'timeClosed', 'viewingTime',
       'itemOrder', 'score', 'ILZ_D', 'ILZ_E', 'ILZ_F', 'ILZ_MA', 'ILZ_N',
       'date', 'year', 'previous_sessions', 'years_from_start', 'pace',
       'frequency', 'IRT_difficulty', 'discrimination', 'matrix', 'topic',
       'task_type', 'item_type', 'difficulty', 'responseformat',
       'numberofchoices', 'audiolength', 'textlength', 'nchar', 'nwords',
       'canton', 'school_postcode', 'school_location', 'day', 'motherTongue',
       'Gender'],
      dtype='object')

In [14]:
if DROPNA:
    df = df[COLS].dropna(subset=MAINCOLS).reset_index()
#print(df.shape)
print(df.isnull().sum())

index                     0
studentId                 0
motherTongue              0
Gender                    0
code                      0
scale                     0
matrix             46032444
IRT_difficulty     46032444
topic              46032444
responseformat     46032444
textlength         46032444
score                     0
age                       0
grade                     0
viewingTime             214
timestamp                 0
useCase                   0
assessmentId              0
school_location    21655164
school_postcode    11651189
canton             11543332
school_code               0
dtype: int64


In [15]:
print(df.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46032444 entries, 0 to 46032443
Data columns (total 22 columns):
 #   Column           Dtype  
---  ------           -----  
 0   index            int64  
 1   studentId        int32  
 2   motherTongue     int8   
 3   Gender           float64
 4   code             object 
 5   scale            object 
 6   matrix           float32
 7   IRT_difficulty   float32
 8   topic            float32
 9   responseformat   float32
 10  textlength       object 
 11  score            float32
 12  age              float32
 13  grade            int8   
 14  viewingTime      float64
 15  timestamp        float32
 16  useCase          object 
 17  assessmentId     object 
 18  school_location  object 
 19  school_postcode  object 
 20  canton           object 
 21  school_code      object 
dtypes: float32(7), float64(2), int32(1), int64(1), int8(2), object(9)
memory usage: 27.7 GB
None


In [16]:
df_aux = df.drop(columns=['index']).drop_duplicates(['studentId', 'assessmentId']).sort_values(['studentId', 'timestamp'], ascending=True)
#previous_timestamps = df_aux.groupby(['studentId']).cumcount().reset_index()
df_aux['previous_sessions'] = df_aux.set_index(['studentId']).groupby(level=[0]).cumcount().values
df_aux['years_from_start'] = df_aux.groupby('studentId', group_keys=False).age.apply(lambda x: x - x.min()).values
df_aux['pace'] = df_aux['years_from_start']/(df_aux['previous_sessions'] + 1e-6)
df_aux['frequency'] = 1/(df_aux['pace'] + 1e-6)

In [17]:
df.set_index(['studentId', 'assessmentId'], inplace=True)
df_aux.set_index(['studentId', 'assessmentId'], inplace=True)
df = df.join(df_aux[['frequency', 'previous_sessions', 'years_from_start']], how='left').reset_index()
EDGE_COLS = EDGE_COLS + ['frequency', 'previous_sessions', 'years_from_start']
COLS = STUDENT_COLS + ITEM_COLS + EDGE_COLS

In [18]:
print(df.info(memory_usage="deep"))
fcols = df.select_dtypes('float').columns
icols = df.select_dtypes('integer').columns
df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')
print(df.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46032444 entries, 0 to 46032443
Data columns (total 25 columns):
 #   Column             Dtype  
---  ------             -----  
 0   studentId          int32  
 1   assessmentId       object 
 2   index              int64  
 3   motherTongue       int8   
 4   Gender             float64
 5   code               object 
 6   scale              object 
 7   matrix             float32
 8   IRT_difficulty     float32
 9   topic              float32
 10  responseformat     float32
 11  textlength         object 
 12  score              float32
 13  age                float32
 14  grade              int8   
 15  viewingTime        float64
 16  timestamp          float32
 17  useCase            object 
 18  school_location    object 
 19  school_postcode    object 
 20  canton             object 
 21  school_code        object 
 22  frequency          float64
 23  previous_sessions  int64  
 24  years_from_start   float32
dtypes: float32(8), f

In [19]:
df.head()

,studentId,assessmentId,index,motherTongue,Gender,code,scale,matrix,IRT_difficulty,topic,...,viewingTime,timestamp,useCase,school_location,school_postcode,canton,school_code,frequency,previous_sessions,years_from_start
0,6842,0D59C4843FFC39195EDE1DD140506CF3DF8BC6F132AAF5...,4,1,0.0,C_P6_FR_LES_A0156_00001_V01,fles,NaN,NaN,NaN,...,34.090,0.0,ms-thema-auto,NaN,NaN,NaN,"c(-""01",1000000.0,1,0.0
1,6842,0D59C4843FFC39195EDE1DD140506CF3DF8BC6F132AAF5...,5,1,0.0,C_P6_FR_LES_A0156_00002_V01,fles,NaN,NaN,NaN,...,29.599,0.0,ms-thema-auto,NaN,NaN,NaN,"c(-""01",1000000.0,1,0.0
2,6842,0D59C4843FFC39195EDE1DD140506CF3DF8BC6F132AAF5...,6,1,0.0,C_P6_FR_LES_A0156_00003_V01,fles,NaN,NaN,NaN,...,9.619,0.0,ms-thema-auto,NaN,NaN,NaN,"c(-""01",1000000.0,1,0.0
3,6842,0D59C4843FFC39195EDE1DD140506CF3DF8BC6F132AAF5...,7,1,0.0,C_P6_FR_LES_A0156_00004_V01,fles,NaN,NaN,NaN,...,21.253,0.0,ms-thema-auto,NaN,NaN,NaN,"c(-""01",1000000.0,1,0.0
4,6842,0D59C4843FFC39195EDE1DD140506CF3DF8BC6F132AAF5...,8,1,0.0,C_P6_FR_LES_A0156_00005_V01,fles,NaN,NaN,NaN,...,15.890,0.0,ms-thema-auto,NaN,NaN,NaN,"c(-""01",1000000.0,1,0.0


In [20]:
# matrize or topics sampling 
# sample one case which has 

def sample_group(df, columns):
    print("* Sampling group")
    if columns == []:
        return df
    columns = ['studentId','assessmentId'] + columns
    df = df.groupby(by=columns).sample(1).reset_index()
    print(df.shape)
    return df

def remove_min(df, MINSTUDENTS, MINITEMS, nsubs=0, balanced=True):
    print("* Removing min")
    print(df.shape)
    # remove items with too few students
    df_student_item = df[['studentId', 'code']].drop_duplicates()

    nstudents = df_student_item.groupby('code').count()
    items_enough_students = nstudents.index[nstudents.studentId > MINSTUDENTS].tolist()

    df = df.loc[df.code.isin(items_enough_students)]

    # remove students with too few items
    df_student_item = df[['studentId', 'code']].drop_duplicates()

    nitems = df_student_item.groupby('studentId').count()
    students_enough_items = nitems.index[nitems.code > MINITEMS].tolist()

    df = df.loc[df.studentId.isin(students_enough_items)]
    
    if nsubs > 0:
        if balanced:
            n_grades = df['grade'].nunique()
            df_list = []
            for grade in df['grade'].unique():
                df_grade = df.loc[df.grade == grade]['studentId'].unique()
                n_subs_grade = np.minimum(nsubs//n_grades, len(df_grade))                 
                studentIds = np.random.choice(df_grade, n_subs_grade, replace=False)    
                df_list.append(df.loc[df.studentId.isin(studentIds)])
                
            df = pd.concat(df_list)
        
        else:
            studentIds = np.random.choice(df['studentId'].unique(), nsubs, replace=False)
            df = df.loc[df.studentId.isin(studentIds)]
            
    print(df.columns)
    print(df.studentId.nunique())
    print(df.matrix.nunique())
    print(df.topic.nunique())    
    print(df.shape)
    return df
        
def get_attributes(df):
    print("* Getting attributes")
    # edge attributes
    df_edge = df[COLS].groupby(by=['studentId', 'code']).agg(mymode).reset_index() 

    # student attributes    
    df_student = df[STUDENT_COLS].loc[df.studentId.isin(df_edge.studentId)].groupby(by=['studentId']).agg(mymode).reset_index() 

    # item attributes
    df_item = df[ITEM_COLS].loc[df.code.isin(df_edge.code)].groupby(by=['code']).agg(mymode).reset_index()
    
    df_edge = df[COLS].groupby(by=['studentId', 'code']).agg(mymode).reset_index()
    
    # recode
    le_student = LabelEncoder()
    le_item = LabelEncoder()

    le_student.fit(df_student['studentId'])
    le_item.fit(df_item['code'])

    df_edge['studentId'] = le_student.transform(df_edge['studentId'])
    df_edge['code'] = le_item.transform(df_edge['code'])

    df_student['studentId'] = le_student.transform(df_student['studentId'])
    df_item['code'] = le_item.transform(df_item['code'])
        
    print(50*'-')
    print('df_student')
    print(df_student.info(memory_usage="deep"))
    print(df_student)

    print(50*'-')
    print('df_item')
    print(df_item.info(memory_usage="deep"))
    print(df_item)

    print(50*'-')
    print('df_edge')
    print(df_edge.info(memory_usage="deep"))
    print(df_edge)
    
    return df_edge, df_item, df_student

def process_data(df, columns, OUT_PATH, NSUBS=0):    
    if columns == []:
        suffix = 'full'
    else:
        suffix = '_'.join(columns)
        
    if NSUBS > 0:
        suffix = suffix + f"_{NSUBS}"
    
    #print(df)
    df = sample_group(df, columns)
    df = remove_min(df, MINSTUDENTS, MINITEMS, NSUBS)
    df_edge, df_item, df_student = get_attributes(df)
    
    plt.figure()
    df_edge.grade.plot.hist()
    plt.show()
    
    df_edge.to_csv(f"{OUT_PATH}_{suffix}.csv")
    print(f"Saving to {OUT_PATH}_{suffix}.csv")
    

In [21]:
%matplotlib inline
    

process_data(df, [], OUT_PATH)
#process_data(df, ['topic'], OUT_PATH)
#process_data(df, ['matrix'], OUT_PATH)

#process_data(df, ['matrix','topic'], OUT_PATH)

#if TEST:
#    OUT_PATH = OUT_PATH + '_test'
#else:
#    process_data(df, [], OUT_PATH, NSUBS)
#    process_data(df, ['topic'], OUT_PATH, NSUBS)
#    process_data(df, ['matrix'], OUT_PATH, NSUBS)
#    process_data(df, ['matrix','topic'], OUT_PATH, NSUBS)

* Sampling group
* Removing min
(46032444, 25)
Index(['studentId', 'assessmentId', 'index', 'motherTongue', 'Gender', 'code',
       'scale', 'matrix', 'IRT_difficulty', 'topic', 'responseformat',
       'textlength', 'score', 'age', 'grade', 'viewingTime', 'timestamp',
       'useCase', 'school_location', 'school_postcode', 'canton',
       'school_code', 'frequency', 'previous_sessions', 'years_from_start'],
      dtype='object')
112852
0
0
(45727626, 25)
* Getting attributes


KeyboardInterrupt: 

In [ ]:
if not TEST:
    NSUBS = 1000
    scale = 'mzuv'
    df_ = df.query(f"scale == '{scale}'")
    df_ = remove_min(df_, MINSTUDENTS, MINITEMS, NSUBS)
    df_edge, df_item, df_student = get_attributes(df_)
    df_edge.to_csv(f"{OUT_PATH}_{scale}_{NSUBS}.csv")
